In [1]:
%matplotlib inline
from os import listdir
from os.path import isfile, join
import numpy as np
import pandas as pd
from  ggplot import *

/home/ramon/anaconda2/lib/python2.7/site-packages/ggplot/utils.py:81: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access Timestamp as pandas.Timestamp
  pd.tslib.Timestamp,
/home/ramon/anaconda2/lib/python2.7/site-packages/ggplot/stats/smoothers.py:4: FutureWarning: The pandas.lib module is deprecated and will be removed in a future version. These are private functions and can be accessed from pandas._libs.lib instead
  from pandas.lib import Timestamp
/home/ramon/anaconda2/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


Read all files names

In [ ]:
col_interest =['id', 'host_id', 'host_since',
       'host_listings_count', 'host_total_listings_count',
       'property_type', 'room_type', 'bathrooms',
       'bedrooms', 'beds', 'price', 'calendar_updated', 'availability_30',
       'availability_60', 'availability_90', 'availability_365',
       'number_of_reviews', 'first_review', 'last_review',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'reviews_per_month']

In [ ]:
def combine_cities():
    local_files =  [join('data/benchmark_capitals/',f) for f in listdir('data/benchmark_capitals/') if isfile(join('data/benchmark_capitals/', f))]
    local_files = [f for f in local_files if "all_" not in f]
    df_all_cities = pd.DataFrame()
    print local_files

    
    for cities in local_files:
        city_name =  cities.split('_')[2].split('.')[0]
        df = pd.read_csv(cities)
        df = df[col_interest]
        df.price = df.price.astype(str)
        df.price= df.price.str.replace('[$,]','')
        df['city']=city_name.title()
        df_all_cities =pd.concat([df_all_cities,df],axis=0)
    return df_all_cities

In [ ]:
df = combine_cities()
df.to_csv('data/benchmark_capitals/all_listings.csv',index=False)

In [ ]:
def is_professional(x):
    return (np.mean((x!=0)&(x<25))*100.0)

In [ ]:
df_out = df.groupby('room_type').agg({'availability_30':{"mean_availability_30":np.mean, 'ratio_professional': is_professional}, "reviews_per_month":{'average_reviews_per_month':'mean'}, 'price':{'mean_price':'mean'}})[['availability_30','reviews_per_month','price']]

In [ ]:
df_out.columns = df_out.columns.droplevel(0)
df_out

In [2]:
df_berlin = pd.read_csv('../../data/benchmark_capitals/listings_berlin.csv')

In [3]:
df_berlin['total_appart_host']=df_berlin.groupby(["host_id"])["id"].transform("count")

In [4]:
df_berlin.total_appart_host.value_counts()

1     16092
2      2560
3       801
4       304
5       140
8       104
9        99
6        96
7        70
12       48
24       48
34       34
32       32
15       30
29       29
11       22
20       20
19       19
18       18
10       10
Name: total_appart_host, dtype: int64

In [5]:
df_berlin.host_listings_count.value_counts()

1.0      15337
2.0       2868
3.0        935
4.0        369
5.0        218
6.0        118
7.0         89
8.0         81
12.0        80
9.0         75
10.0        38
15.0        36
0.0         35
34.0        34
45.0        32
29.0        29
18.0        27
28.0        24
33.0        24
11.0        24
27.0        20
19.0        19
13.0        17
17.0        10
14.0         9
46.0         3
32.0         2
16.0         2
22.0         1
23.0         1
180.0        1
Name: host_listings_count, dtype: int64

In [ ]:
df_berlin[['total_appart_host','host_listings_count']].head(200).plot.bar()

In [ ]:
pd.to_datetime(df_berlin.host_since).hist()

In [ ]:
(pd.to_datetime(df_berlin.host_since)).idxmin()

In [ ]:
df_berlin.iloc[18147]

In [ ]:
df_berlin.host_since.max()

In [ ]:
np.mean(df_berlin.availability_30.isnull())

In [ ]:
df_berlin['occupancy_rate'] = (30-df_berlin[df_berlin.availability_30>0].availability_30)*100.0//30.0
df_berlin.occupancy_rate.hist()

In [ ]:
ggplot(df_berlin,aes(x='occupancy_rate',y='reviews_per_month'))+ geom_boxplot()

In [ ]:
df_berlin.last_review = pd.to_datetime(df_berlin.last_review)

In [ ]:
t_zero =df_berlin.last_review.max()

In [ ]:
df_berlin['days_since_last_review']= (t_zero-df_berlin.last_review).dt.days

In [ ]:
df_berlin['is_active_host']= df_berlin.days_since_last_review<10

In [ ]:
df_berlin.groupby('room_type')[['is_active_host']].mean()

In [ ]:
pd.DataFrame(df_berlin.isnull().sum()/len(df_berlin)*100,columns=['ratio_missing']).sort_values(by=)